In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from classifier import Data

dataset = Data(
    data_path = "../data/imdb.train.sample.csv",
    polarities = {
      "negative": 0,
      "positive": 1
    },
    data_label='review',
    target_label='sentiment'
)

In [4]:
from classifier._neural import Encoder

enc = Encoder({
    'model': 'fabriceyhc/bert-base-uncased-imdb',
    'layers': [11]
})

In [5]:
enc.df_encode(dataset.data, col=dataset.data_label, label=dataset.data_path)

In [6]:
import numpy as np
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return MDS(n_components=dim).fit_transform(data)

In [7]:
from typing import Generator
import torch
import numpy as np
import pandas as pd
from scipy.spatial import distance


def analysis(
        data: pd.DataFrame,
        dim: list = None
) -> Generator:
    if dim is None:
        dim = [768, 576, 384, 192, 96, 48, 24, 12, 6, 3]

    embed_col: np.ndarray = torch.stack(data[enc.col_name].tolist()).numpy()

    for d in dim:

        # create record to keep row data
        record: dict = {'dim': d}

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == enc.dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        # compute centroid means and dispersion for each cluster
        for label, group in data.groupby(dataset.target_label):
            record[f'centroid_{label}'] = np.mean(np.stack(group['reduced_embeds'].tolist(), axis=0), axis=0).tolist()
            record[f'dispersion_{label}'] = np.sum(distance.pdist(group['reduced_embeds'].tolist()))

        record['distance'] = distance.cdist([record['centroid_positive']], [record['centroid_negative']]).item(0)

        yield record

In [8]:
import pandas as pd

cluster_analysis = pd.DataFrame.from_records(list(analysis(dataset.data)))
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']]

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.665069e+06,1.069949e+06,1.251342
1,576,1.665929e+06,1.070749e+06,1.189003
2,384,1.665837e+06,1.070677e+06,1.188894
3,192,1.665245e+06,1.070195e+06,1.214624
4,96,1.664408e+06,1.069640e+06,1.216426
5,48,1.661735e+06,1.067368e+06,1.310085
6,24,1.656986e+06,1.063510e+06,1.414011
7,12,1.646610e+06,1.055193e+06,1.569133
8,6,1.622480e+06,1.036313e+06,1.808590
9,3,1.583517e+06,1.001442e+06,1.307229


In [9]:
import pandas as pd

cluster_analysis = pd.DataFrame.from_records(list(analysis(dataset.data)))
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']]

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.665069e+06,1.069949e+06,1.251342
1,576,1.665990e+06,1.070803e+06,1.184022
2,384,1.665881e+06,1.070745e+06,1.183920
3,192,1.665307e+06,1.070273e+06,1.206436
4,96,1.664282e+06,1.069415e+06,1.232434
5,48,1.661879e+06,1.067403e+06,1.309405
6,24,1.656779e+06,1.063651e+06,1.410591
7,12,1.644861e+06,1.053681e+06,1.685063
8,6,1.626873e+06,1.039705e+06,1.586060
9,3,1.578766e+06,9.972874e+05,1.550226


In [10]:
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']].to_csv('./cluster_analysis.csv')